In [1]:
!pip install konlpy

In [2]:

import sys
!{sys.executable} -m pip install ekonlpy

You should consider upgrading via the 'c:\Users\eeyy1\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
##데이터 필터링(2012년 이후 파일만 재수집)
import os
import shutil

# 1. 원본 파일이 있는 폴더 경로 (본인의 경로로 수정하세요)
source_dir = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\bond_text_final_v5'

# 2. 2012년 이후 파일만 따로 모을 새 폴더 경로
target_dir = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\bond_text_2012_to_2025'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# 3. 기준 날짜 설정
start_date = "20120101"

print("🔍 파일 필터링을 시작합니다...")

count = 0
for filename in os.listdir(source_dir):
    # 파일명 앞 8자리가 날짜라고 가정 (예: 20120115_증권사_제목.txt)
    file_date = filename[:8]
    
    # 숫자로 된 날짜인지 확인하고 기준일보다 크거나 같으면 복사
    if file_date.isdigit() and file_date >= start_date:
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        
        shutil.copy2(src_path, dst_path) # 파일 복사 (원본 유지)
        count += 1

print(f"✅ 필터링 완료! {count}개의 파일이 '{target_dir}'로 복사되었습니다.")

🔍 파일 필터링을 시작합니다...
✅ 필터링 완료! 7722개의 파일이 'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\bond_text_2012_to_2025'로 복사되었습니다.


In [4]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\eeyy1\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import os
import re
import pandas as pd
from tqdm import tqdm

# 1. 파일 경로 설정
source_dir = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\bond_text_2012_to_2025'
save_path = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\integrated_reports_2012_2025.csv'

# --- [수정된 함수 1] 텍스트 정제 (숫자와 마침표 보존) ---

def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    # 1. 저작권 및 불필요 문구 제거 (이전과 동일)
    boilerplate_patterns = [
        r"본 자료에 수록된.*", r"본 자료는 어떠한.*", r"Copyright.*", 
        r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", 
        r"https?://\S+|www\.\S+", r"Tel:.*",
    ]
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, " ", text)

    # 2. [중요] 줄바꿈(\n)이나 탭(\t)을 마침표와 공백으로 치환
    # 마침표 없이 줄바꿈만 된 문장들을 강제로 분리해줍니다.
    text = text.replace('\n', '. ').replace('\t', ' ')

    # 3. 한글, 숫자, 공백, 마침표(.)만 남기기
    text = re.sub(r'[^가-힣0-9\s\.]', ' ', text)
    
    # 4. 연속된 마침표(.. )나 공백 정리
    text = re.sub(r'\.+', '.', text) # .. -> .
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# --- [새로 추가된 함수 2] 소수점 보호 문장 분리 ---
def split_sentences(text):
    # 소수점(3.25)은 무시하고, 한글 뒤에 오는 마침표만 문장의 끝으로 인식
    # 이 로직이 있어야 나중에 문장별 점수를 매길 때 정확해집니다.
    sentence_split_pattern = r'(?<=[가-힣])\.(?=\s|[가-힣]|$)'
    sentences = re.split(sentence_split_pattern, text)
    return [sent.strip() + "." for sent in sentences if len(sent.strip()) > 5]

# 2. 날짜별로 텍스트 통합하기
data_dict = {}

print(f"📂 폴더 내 파일 읽기 시작: {source_dir}")
filenames = [f for f in os.listdir(source_dir) if f.endswith('.txt')]

for filename in tqdm(filenames, desc="통합 중"):
    file_date = filename[:8]
    file_path = os.path.join(source_dir, filename)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        cleaned_content = clean_text(content)
        
        if cleaned_content:
            if file_date not in data_dict:
                data_dict[file_date] = cleaned_content
            else:
                data_dict[file_date] += " " + cleaned_content
                
    except Exception as e:
        print(f"❌ 오류 발생 ({filename}): {e}")

# 3. 결과 저장
if data_dict:
    df = pd.DataFrame(list(data_dict.items()), columns=['date', 'content'])
    df = df.sort_values(by='date').reset_index(drop=True)
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    
    print("\n" + "="*50)
    print(f"✅ 작업 완료! 소수점이 보호된 데이터가 저장되었습니다.")
    print(f"📊 총 통합 날짜 수: {len(df)}일")
    print("="*50)

📂 폴더 내 파일 읽기 시작: C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\bond_text_2012_to_2025


통합 중:   0%|          | 0/7722 [00:00<?, ?it/s]

통합 중: 100%|██████████| 7722/7722 [00:09<00:00, 793.03it/s] 



✅ 작업 완료! 소수점이 보호된 데이터가 저장되었습니다.
📊 총 통합 날짜 수: 2466일


In [7]:
##n-gram추출
import sys
import os

# eKoNLPy가 설치된 경로를 직접 추가
lib_path = r'C:\Users\eeyy1\AppData\Local\Programs\Python\Python311\Lib\site-packages'
if lib_path not in sys.path:
    sys.path.append(lib_path)

try:
    from ekonlpy.sentiment import MPCK
    print("✅ eKoNLPy 로드 성공!")
except ModuleNotFoundError:
    print("❌ 여전히 찾을 수 없습니다. 경로를 다시 확인해주세요.")


# 1. eKoNLPy의 금융 통화정책 분석기 초기화
mpck = MPCK()

def get_sentiment_features(text):
    # 2. 텍스트를 토큰화 (금융 단어 및 n-gram 추출)
    tokens = mpck.tokenize(text)
    
    # 3. 매파(Hawkish)/비둘기파(Dovish) 판단을 위한 n-gram 리스트 추출
    ngram = mpck.ngramize(tokens)
    
    return tokens + ngram

# 예시: "금리가 인상될 것으로 보입니다." -> ['금리/NNG', '인상/NNG', '금리;인상']


✅ eKoNLPy 로드 성공!


In [8]:
from ekonlpy.tag import Mecab

# MPCK 대신 Mecab 형태소 분석기를 바로 사용합니다.
# eKoNLPy의 Mecab은 금융 용어 사전이 내장되어 있습니다.
mecab = Mecab()

def get_tokens(text):
    # 금융 용어 단위로 토큰화
    return mecab.pos(text)

# 테스트
test_text = "금리 인상 가능성이 높습니다."
print(get_tokens(test_text))

[('금리', 'NNG'), ('인상', 'NNG'), ('가능성', 'NNG'), ('이', 'JKS'), ('높', 'VA'), ('습니다', 'EF'), ('.', 'SF')]


In [10]:
import pandas as pd
import os
from ekonlpy.sentiment import MPCK
from tqdm import tqdm

# 1. 경로 설정 (따옴표 오류 수정 완료)
csv_path = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\integrated_reports_2012_2025.csv'
output_dir = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\preprocessing\bond_reports_preprocessing'
output_path = os.path.join(output_dir, 'tokenized_reports_final.csv')

# 저장 폴더가 없으면 자동 생성
os.makedirs(output_dir, exist_ok=True)

# 2. 데이터 불러오기
print("📂 데이터를 불러오는 중...")
df = pd.read_csv(csv_path)
print(f"✅ 총 {len(df)}개의 리포트 로드 완료.")

# 3. eKoNLPy 금융 형태소 분석기(MPCK) 초기화
# ※ 만약 여기서 UnicodeDecodeError가 난다면 아까 알려드린 mpck.py 수정을 꼭 해주세요!
mpck = MPCK()

def get_tokens_and_ngrams(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return [] # 빈 문자열 대신 빈 리스트 반환
    
    try:
        tokens = mpck.tokenize(text)
        ngrams = mpck.ngramize(tokens)
        
        # 리스트와 리스트를 더해서 하나의 리스트로 만듭니다.
        result = tokens + ngrams
        
        return result  # [", ".join(result) 대신 리스트를 직접 반환]
    except Exception as e:
        return []

# 4. 전체 데이터 분석 시작
print(f"🚀 분석 시작 (예상 소요 시간: 약 30~60분)...")
tqdm.pandas() # 진행바 설정

# 'content' 컬럼의 텍스트를 분석하여 'tokens' 컬럼에 저장
df['tokens'] = df['content'].progress_apply(get_tokens_and_ngrams)

# 5. 결과 저장 (날짜와 토큰화 결과만 깔끔하게 저장)
# 후에 금리 데이터와 병합하기 위해 date와 tokens만 추출합니다.
final_save_df = df[['date', 'tokens']].copy()
final_save_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("-" * 50)
print(f"✅ 모든 작업이 완료되었습니다!")
print(f"💾 저장 위치: {output_path}")
print(f"📊 최종 데이터 수: {len(final_save_df)}개")

📂 데이터를 불러오는 중...
✅ 총 2466개의 리포트 로드 완료.
🚀 분석 시작 (예상 소요 시간: 약 30~60분)...


100%|██████████| 2466/2466 [13:03<00:00,  3.15it/s]


--------------------------------------------------
✅ 모든 작업이 완료되었습니다!
💾 저장 위치: C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\preprocessing\bond_reports_preprocessing\tokenized_reports_final.csv
📊 최종 데이터 수: 2466개


In [11]:
import pandas as pd
from ekonlpy.sentiment.utils import MPTokenizer
from tqdm import tqdm
import os

# 1. 경로 설정 (따옴표 중복 제거 버전)
base_path = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\preprocessing\bond_reports_preprocessing'
token_csv_path = os.path.join(base_path, 'tokenized_reports.csv')

# 2. 파일 읽기
df = pd.read_csv(token_csv_path)
tokenizer = MPTokenizer()

def make_ngrams(tokens_str):
    if not isinstance(tokens_str, str) or tokens_str == "":
        return ""
    token_list = [t.strip() for t in tokens_str.split(',')]
    try:
        # 논문 핵심: 단어들을 n-gram으로 조합 (예: 금리/NNG;인상/NNG)
        ngrams = tokenizer.ngramize(token_list)
        return ", ".join(token_list + ngrams)
    except:
        return tokens_str

print("🚀 논문 규격 n-gram 생성 시작...")
tqdm.pandas()
df['ngram_features'] = df['tokens'].progress_apply(make_ngrams)

# 3. 중간 파일 저장 (이게 있어야 다음 단계가 돌아갑니다)
feature_path = os.path.join(base_path, 'final_paper_features.csv')
df.to_csv(feature_path, index=False, encoding='utf-8-sig')
print(f"✅ n-gram 추출 및 저장 완료: {feature_path}")

🚀 논문 규격 n-gram 생성 시작...


100%|██████████| 2458/2458 [00:39<00:00, 61.82it/s] 


✅ n-gram 추출 및 저장 완료: C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\preprocessing\bond_reports_preprocessing\final_paper_features.csv


In [12]:
# 1. 원본 리포트와 위에서 만든 전처리 파일 경로
raw_path = r'C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\crawler\bond_reports\integrated_reports_2012_2025.csv'
feature_path = os.path.join(base_path, 'final_paper_features.csv')

# 2. 데이터 로드 및 날짜 통일
df_raw = pd.read_csv(raw_path)
df_features = pd.read_csv(feature_path)

df_raw['date'] = pd.to_datetime(df_raw['date'].astype(str)).dt.strftime('%Y-%m-%d')
df_features['date'] = pd.to_datetime(df_features['date'].astype(str)).dt.strftime('%Y-%m-%d')

# 3. 필요한 컬럼만 추출 (content와 tokens만 남기기)
df_raw_sub = df_raw[['date', df_raw.columns[1]]].copy() 
df_raw_sub.columns = ['date', 'content']

df_features_sub = df_features[['date', 'ngram_features']].copy()
df_features_sub.columns = ['date', 'tokens']

# 4. 날짜 기준 병합 및 중복 날짜 합치기
final_df = pd.merge(df_raw_sub, df_features_sub, on='date', how='inner')
final_df = final_df.groupby('date').agg({
    'content': lambda x: ' '.join(x.astype(str)),
    'tokens': lambda x: ', '.join(x.astype(str))
}).reset_index()

# 5. 최종 형식 맞추기 및 저장
final_df['category'] = "리포트"
final_df['source'] = "증권사"
final_df = final_df[['date', 'content', 'tokens', 'category', 'source']]

final_output = os.path.join(base_path, 'final_integrated_full_v2.csv')
final_df.to_csv(final_output, index=False, encoding='utf-8-sig')

print(f"🎉 모든 작업 완료! 최종 파일: {final_output}")
print(f"📊 총 {len(final_df)}일치의 데이터셋이 준비되었습니다.")

🎉 모든 작업 완료! 최종 파일: C:\KDT\only-pull-me\PJT_BOK\team2_forecast_pjt\preprocessing\bond_reports_preprocessing\final_integrated_full_v2.csv
📊 총 2458일치의 데이터셋이 준비되었습니다.


In [13]:
import pandas as pd

# 1. 파일 불러오기 (용량이 크므로 상위 5개만)
df_sample = pd.read_csv(final_output, nrows=5) 

# 2. 마침표 존재 여부 체크
for i, content in enumerate(df_sample['content']):
    has_dot = "." in content
    print(f"[{i}번 행] 마침표 포함 여부: {has_dot}")
    if has_dot:
        # 마침표 주변 20글자 출력해서 확인
        dot_idx = content.find(".")
        print(f"   샘플 문구: ...{content[max(0, dot_idx-10):dot_idx+10]}...")

[0번 행] 마침표 포함 여부: True
   샘플 문구: ... 바라기 이른 시점. 2012년 1월...
[1번 행] 마침표 포함 여부: True
   샘플 문구: ...13 2012. 2012년 1월...
[2번 행] 마침표 포함 여부: True
   샘플 문구: ...보며 대응 할 필요. 2012년 1월...
[3번 행] 마침표 포함 여부: True
   샘플 문구: .... 2012년 1월...
[4번 행] 마침표 포함 여부: True
   샘플 문구: ...펀더멘탈의 오버랩 . 2012년 1월...


In [14]:
# 문장 형태가 잘 보존되었는지 더 긴 샘플 확인
sample_text = df_sample['content'].iloc[1] # 1번 인덱스(두 번째 날짜) 데이터

# 마침표를 기준으로 잘랐을 때 문장이 말이 되는지 확인
sentences = sample_text.split('.')
print("--- 분리된 문장 샘플 (상위 3개) ---")
for sent in sentences[:3]:
    print(f"문장: {sent.strip()}.")

--- 분리된 문장 샘플 (상위 3개) ---
문장: 13 2012.
문장: 2012년 1월 금융통화위원회.
문장: 성장.
